<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../_static/logo_stacked.png" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 8. Scaling</h2></div>
<br><br>